<a href="https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/part-b-estimation-using-lang-features/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Study 1: Designing explainable speech-based machine learning models of depression

To open this ipynb in Colab, click the "Open in Colab" button at the top of the ipynb in Github, or [follow this link](https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/main/main.ipynb).

Given that Colab doesn't automatically load any of the content (data or other functions) from the Github repo, running the code below will copy the repo into the workspace directory for use. To save this ipynb file back to Github, select **File > Save** (which should show the repo if you're signed in) or **File > Save a copy in Github** if it's in the menu.

Note that the content of the data files or any of the other file structures are not saved back to Github, so make sure that if you make changes to things there, that you put them in Github separately.

In [ ]:
# Clone Github Repo into the temporary local environment so data can be accessed and manipulated
!git clone https://github.com/cgenevier/CSCI5622-HW4.git
%cd CSCI5622-HW4

In [ ]:
# Import necessary libraries

# Helpers
import glob

# Pandas, seaborn, and numpy for data manipulation
import pandas as pd
pd.set_option("display.max_rows", None)
import statistics as stat
import seaborn as sns
import numpy as np
np.random.seed(42)

# Keras & TensorFlow for building the neural networks
import itertools, json, time
from itertools import count
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, callbacks, backend as K
tf.random.set_seed(42)

# Feature extraction
!pip install vaderSentiment transformers torch
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import logging, BertTokenizer, BertModel
import torch

# Matplotlib for graphing
import matplotlib.pyplot as plt

# Disable progress bars (necessary for it to show up correctly in Github)
logging.set_verbosity_error()

##### Formatting the data - Depression Labels

In [ ]:
# Import Depression Labels
# Columns: Participant_ID, PHQ_Score
depression_labels = pd.read_csv("data/DepressionLabels.csv")

# Rename Participant_ID to ParticipantID to match accoustic files below & force trimmed string type
depression_labels = depression_labels.rename(columns={"Participant_ID": "ParticipantID"})
depression_labels["ParticipantID"] = depression_labels["ParticipantID"].astype(str).str.strip()

##### Formatting the data - Text Features

In [ ]:
# Import Text Dataset (for text feature extraction)
# Note: When comparing the E-DAIC_Transcripts files to the corresponding E-DAIC Acoustics files,
# it looks like the transcripts sometimes only contain partial data from the accoustics text column -
# for example, 386_Transcript.csv - so it seems to make sense to concatenate Text data in the
# Acoustics file for completeness.
rows = []
for p in glob.glob("data/E-DAIC_Acoustics/*_utterance_agg.csv"):
    df = pd.read_csv(p)
    df["ParticipantID"] = df["ParticipantID"].astype(str).str.strip()
    full_text = " ".join(df["Text"].dropna().astype(str))
    full_text = " ".join(full_text.split())  # collapse whitespace
    rows.append({"ParticipantID": df["ParticipantID"].iloc[0], "FullText": full_text})

# Columns: ParticipantID, FullText
text_df = pd.DataFrame(rows)
# Merge with labels. Columns: ParticipantID, FullText, PHQ_Score
lang_df = depression_labels.merge(text_df, on="ParticipantID", how="inner")

# Inspect results
lang_df.head()

##### Formatting the data - Acoustic Features

In [ ]:
# Import Accoustic Dataset (for part c, d)

# Helper function for mean, standard dev, & IQR (interquartile range)
def summarize_cols(num_df: pd.DataFrame) -> pd.DataFrame:
    # mean and std per column (NaN-safe)
    mean_s = num_df.mean(numeric_only=True)
    std_s  = num_df.std(numeric_only=True)
    # IQR = Q3 - Q1 per column
    q75 = num_df.quantile(0.75, numeric_only=True)
    q25 = num_df.quantile(0.25, numeric_only=True)
    iqr_s = q75 - q25

    # assemble into a tidy (feature, stat) table
    stats = pd.concat(
        {"mean": mean_s, "std": std_s, "iqr": iqr_s},
        axis=1
    )  # index=feature, columns=[mean,std,iqr]

    # flatten to one row with columns like feature__mean
    wide = stats.stack().to_frame().T
    wide.columns = [f"{feat}__{stat}" for feat, stat in wide.columns]
    return wide

# Each file in E-DAIC_Acoustics contains utterance-level acoustic features for one participant.
rows_with_conf = []
rows_no_conf = []
for p in glob.glob("data/E-DAIC_Acoustics/*_utterance_agg.csv"):
    df = pd.read_csv(p)
    df["ParticipantID"] = df["ParticipantID"].astype(str).str.strip()

    # Include Confidence (column 5) + all acoustic features (6+)
    numeric_with_conf = df.columns[5:]
    df[numeric_with_conf] = df[numeric_with_conf].apply(pd.to_numeric, errors="coerce")

    agg_with_conf = summarize_cols(df[numeric_with_conf])
    agg_with_conf.insert(0, "ParticipantID", df["ParticipantID"].iloc[0])
    rows_with_conf.append(agg_with_conf)

    # Excludes Confidence - only include acoustic features
    numeric_no_conf = df.columns[6:]
    df[numeric_no_conf] = df[numeric_no_conf].apply(pd.to_numeric, errors="coerce")

    agg_no_conf = summarize_cols(df[numeric_no_conf])
    agg_no_conf.insert(0, "ParticipantID", df["ParticipantID"].iloc[0])
    rows_no_conf.append(agg_no_conf)

# Combine into one DataFrame each
acoustic_features_with_conf = pd.concat(rows_with_conf, ignore_index=True)
acoustic_features_no_conf = pd.concat(rows_no_conf, ignore_index=True)

# Merge with labels to add PHQ_Score
acoustic_df_with_confidence = depression_labels.merge(acoustic_features_with_conf, on="ParticipantID", how="inner")
acoustic_df_no_confidence   = depression_labels.merge(acoustic_features_no_conf, on="ParticipantID", how="inner")

# Reorder columns: ParticipantID, PHQ_Score, then features
cols = ["ParticipantID", "PHQ_Score"] + [c for c in acoustic_df_with_confidence.columns if c not in ["ParticipantID", "PHQ_Score"]]
acoustic_df_with_confidence = acoustic_df_with_confidence[cols]
cols = ["ParticipantID", "PHQ_Score"] + [c for c in acoustic_df_no_confidence.columns if c not in ["ParticipantID", "PHQ_Score"]]
acoustic_df_no_confidence = acoustic_df_no_confidence[cols]

# Inspect results
display(acoustic_df_with_confidence.head())
display(acoustic_df_no_confidence.head())

### (a) (2 points) Extracting language features.

**Syntactic vectorizers:** count vectorizer (e.g., CountVectorizer from sklearn) transforming
a collection of text documents into a numerical matrix of word or token counts; TFIDF vectorizer (e.g., TfidfVectorizer from sklearn) incorporating document-level weighting,
which emphasizes words significant to specific documents’ part-of-speech features counting
the distribution of part of speech tags over a document

In [ ]:
# Use TfidfVectorizer from sklearn
vect = TfidfVectorizer(max_features=1000)
X_tfidf = vect.fit_transform(lang_df["FullText"])

# Convert sparse matrix to DataFrame
syntactic_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vect.get_feature_names_out()
)

# Add ParticipantID column & move to first column
syntactic_df["ParticipantID"] = lang_df["ParticipantID"].values
cols = ["ParticipantID"] + [c for c in syntactic_df.columns if c != "ParticipantID"]
syntactic_df = syntactic_df[cols]

# Add back in PHQ_Score & move to second column
syntactic_df = syntactic_df.merge(depression_labels, on="ParticipantID", how="inner")
cols = ["ParticipantID", "PHQ_Score"] + [c for c in syntactic_df.columns if c not in ["ParticipantID", "PHQ_Score"]]
syntactic_df = syntactic_df[cols]

# Inspect dataframe
syntactic_df.head()

**Semantic features:** sentiment scores (e.g., Vader, https://github.com/cjhutto/vaderSentiment),
topic distribution (using topic modeling), or named entities

In [ ]:
# Using Vader to analyze sentiment of the text data
analyzer = SentimentIntensityAnalyzer()

# Apply Vader to the text data (creates 4 new columns)
vader_scores = lang_df["FullText"].apply(lambda x: pd.Series(analyzer.polarity_scores(str(x))))
semantic_df = pd.concat([lang_df, vader_scores], axis=1)

# Inspect dataframe
semantic_df.head()


**Advanced features:** word embeddings, such as Word2Vec or BERT (e.g., pytorch-pretrainedbert)) for capturing contextual meaning

In [ ]:
# Use BERT to capture contextual meaning (note: takes about 4 minutes to run on T4)

# Load uncased base model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

# Loop through text data and get embeddings
embeddings = []
for text in lang_df["FullText"]:
    # Truncate long text (BERT max = 512 tokens)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # [CLS] token
    embeddings.append(cls_embedding)

# Convert list of embeddings (each 768-dim) to DataFrame
bert_df = pd.DataFrame(np.vstack(embeddings))
bert_df.columns = [f"bert_{i}" for i in range(bert_df.shape[1])]

# Add ParticipantID and PHQ_Score
bert_df = pd.concat([lang_df[["ParticipantID", "PHQ_Score"]].reset_index(drop=True), bert_df], axis=1)

# Inspect dataframe
bert_df.head()

**Combined dataset:** Combined the three dataframes above into one with all the features

In [ ]:
# Merge all three on ParticipantID
text_feature_df = (
    syntactic_df
    .merge(semantic_df, on=["ParticipantID", "PHQ_Score"], how="outer")
    .merge(bert_df, on=["ParticipantID", "PHQ_Score"], how="outer")
)
text_feature_df.head()

# Optional: sort by ParticipantID for clarity
#merged_df = merged_df.sort_values("ParticipantID").reset_index(drop=True)

### (b) (2 points) Estimating depression severity with interpretable models using language features.

Question: In semantic feature extraction above, there are four features generated: neg, neu, pos, compound. They are inter-related because neg is the proportion of the document that is negative, neu is the proportion of the document that is neutral, pos is the proportion of the document that is positive, and compound is a normalized sentiment value that takes into account all 3. Should we remove some of these features because they're redundant?

In [ ]:
# -------- Part (b): Estimating depression severity using language (VADER) features -------- #

import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import pearsonr
import pandas as pd

# Language features extracted in Part (a)
lang_features = ["neg", "neu", "pos", "compound"]

# Only keep valid rows
df_b = semantic_df.dropna(subset=["PHQ_Score"])

X = df_b[lang_features].values
y = df_b["PHQ_Score"].values

kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = []

# Test different values of k (tree depth)
for k in [2, 3, 4, 5, 6]:
    all_preds = []
    all_truth = []

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Decision tree with varying depth k
        model = DecisionTreeRegressor(max_depth=k, random_state=42)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        all_preds.extend(preds)
        all_truth.extend(y_test)

    all_preds = np.array(all_preds)
    all_truth = np.array(all_truth)

    # Evaluation metrics
    r, _ = pearsonr(all_preds, all_truth)
    RE = np.mean(np.abs(all_preds - all_truth) / np.max(all_truth))

    results.append({"k": k, "Pearson_r": r, "RE": RE})

# Show results table
results_df = pd.DataFrame(results)
display(results_df)

# Train final model with best k (optional)
best_k = results_df.iloc[results_df["Pearson_r"].idxmax()]["k"]
print("\nBest k based on Pearson r:", best_k)


### (c) (2 points) Estimating depression severity with interpretable models using acoustic features.

### (d) (2 points) Estimating depression severity with unimodal and multimodal deep learning models.

Question: In semantic feature extraction above, there are four features generated: neg, neu, pos, compound. They are inter-related because neg is the proportion of the document that is negative, neu is the proportion of the document that is neutral, pos is the proportion of the document that is positive, and compound is a normalized sentiment value that takes into account all 3. Should we remove some of these features because they're redundant?

### (e) (2 points) Explainable ML.

### (f) (Bonus, 2 points) Experimenting with transformers.